In [1]:
import glob
import os
import subprocess

def convert(n, d):
    get_ipython().system("jupyter nbconvert {} --to markdown --output {}".format(n, d))
    subprocess.check_call(["jupyter", "nbconvert", n, "--to", "markdown", "--output", d])
    
for subdir in glob.glob("./sem*"):
    notebooks = glob.glob(subdir + "/*.ipynb")
    print(subdir, notebooks)
    for m in glob.glob(subdir + "/*.md"):
        os.remove(m)
        
#     for n in notebooks:
#         subprocess.check_call(["jupyter", "nbconvert", n, "--to", "pdf", 
#                                "--output", os.path.basename(n.replace(".ipynb", ""))])

    if len(notebooks) == 1:
        convert(notebooks[0], "README")
    else:
        for n in notebooks:
            convert(n, os.path.basename(n.replace(".ipynb", "")))
        


./sem04-asm-arm ['./sem04-asm-arm/arm.ipynb']
[NbConvertApp] Converting notebook ./sem04-asm-arm/arm.ipynb to markdown
[NbConvertApp] Writing 7422 bytes to ./sem04-asm-arm/README.md
./sem11-mmap-instrumentation ['./sem11-mmap-instrumentation/mmap_and_instrumentation.ipynb']
[NbConvertApp] Converting notebook ./sem11-mmap-instrumentation/mmap_and_instrumentation.ipynb to markdown
[NbConvertApp] Writing 23784 bytes to ./sem11-mmap-instrumentation/README.md
./sem10-file-attributes ['./sem10-file-attributes/file-attrib.ipynb']
[NbConvertApp] Converting notebook ./sem10-file-attributes/file-attrib.ipynb to markdown
[NbConvertApp] Writing 16319 bytes to ./sem10-file-attributes/README.md
./sem03-ints-floats ['./sem03-ints-floats/sem3_floats.ipynb', './sem03-ints-floats/sem3_ints.ipynb']
[NbConvertApp] Converting notebook ./sem03-ints-floats/sem3_floats.ipynb to markdown
[NbConvertApp] Writing 2857 bytes to ./sem03-ints-floats/sem3_floats.md
[NbConvertApp] Converting notebook ./sem03-ints-floa

In [2]:
" ".join(['jupyter', 'nbconvert', './sem04-asm-arm/arm.ipynb', '--to', 'pdf', '--output', 'arm'])

'jupyter nbconvert ./sem04-asm-arm/arm.ipynb --to pdf --output arm'

In [3]:
# !jupyter nbconvert ./sem04-asm-arm/arm.ipynb --to pdf --output arm


In [4]:
for sfx in [".ipynb", ".md"]:
    for fname in glob.glob("./sem*/*" + sfx):
        with open(fname, "r") as f:
            r = f.read()
        for b in ["\x00", "\x1B", "\x08"]:
            r = r.replace(b, "")
        with open(fname, "w") as f:
            f.write(r)
        get_ipython().system("dos2unix {}".format(fname))

dos2unix: converting file ./sem04-asm-arm/arm.ipynb to Unix format ...
dos2unix: converting file ./sem11-mmap-instrumentation/mmap_and_instrumentation.ipynb to Unix format ...
dos2unix: converting file ./sem10-file-attributes/file-attrib.ipynb to Unix format ...
dos2unix: converting file ./sem03-ints-floats/sem3_floats.ipynb to Unix format ...
dos2unix: converting file ./sem03-ints-floats/sem3_ints.ipynb to Unix format ...
dos2unix: converting file ./sem07-asm-x86-x87-sse/sse_x86.ipynb to Unix format ...
dos2unix: converting file ./sem07-asm-x86-x87-sse/floating_point_x86.ipynb to Unix format ...
dos2unix: converting file ./sem12-fork-exec-pipe/fork-pipe-exec.ipynb to Unix format ...
dos2unix: converting file ./sem08-asm-x86-nostdlib/nostdlib.ipynb to Unix format ...
dos2unix: converting file ./sem06-asm-x86/asm_x86.ipynb to Unix format ...
dos2unix: converting file ./sem05-asm-arm-addressing/adressing.ipynb to Unix format ...
dos2unix: converting file ./sem01/sem1.ipynb to Unix format

In [5]:
for subdir in glob.glob("./sem*"):
    for sfx in [".ipynb", ".md", ".c", ".cpp"]:
        get_ipython().system("git add {}/*{}".format(os.path.basename(subdir), sfx))
get_ipython().system("git add -u")
get_ipython().system("git commit -m 'yet another update'")
get_ipython().system("git push origin master")

The file will have its original line endings in your working directory.
The file will have its original line endings in your working directory.
fatal: pathspec 'sem04-asm-arm/*.cpp' did not match any files
The file will have its original line endings in your working directory.
The file will have its original line endings in your working directory.
The file will have its original line endings in your working directory.
The file will have its original line endings in your working directory.
fatal: pathspec 'sem10-file-attributes/*.cpp' did not match any files
The file will have its original line endings in your working directory.
The file will have its original line endings in your working directory.
The file will have its original line endings in your working directory.
The file will have its original line endings in your working directory.
fatal: pathspec 'sem03-ints-floats/*.cpp' did not match any files
The file will have its original line endings in your working directory.
The file w

In [6]:
hex(ord('a'))

'0x61'

In [7]:
"\x61"

'a'